<a href="https://colab.research.google.com/github/jogg1989/MCAPTI1Tarea2/blob/main/Tarea2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practica 2

**Nombre:** Jorge Osvaldo Gonzalez Gonzalez  
**e-mail:** jorge.gonzalez@alumnos.udg.mx

### MODULES

In [ ]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

### CLASES

In [ ]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

### FUNCTIONS

In [ ]:
###############################################################################################
# Turning angle
###############################################################################################
def turning_angle(vec_a, vec_b, vec_c):
    """
    Arguments:
        vec_a: First detection coordinates
        vec_b: Second detection coordinates
        vec_c: Third detection coordinates
    Returns:
        theta: Turning angle 
    """
    ab = vec_b-vec_a
    norm_ab = np.linalg.norm(ab)
    
    bc = vec_c-vec_b
    norm_bc = np.linalg.norm(bc)

    dot_p = np.dot(ab, bc)
    
    cross_p = np.cross(ab, bc)
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1
    
    cos_theta = dot_p/(norm_ab*norm_bc+np.finfo(float).eps)
    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta

## Actividad 1: Path Length - (BM1 vs BM2 vs CRW)

* Cargar trayectorias en **Pandas** Data Frame
* Calcular métrica utilizando exclusivamente funciones de NumPy
* Guardar métricas en **Pandas** Data Frame
* Visualizar con **plotly**

In [ ]:
# Load existing trajectories
# BM speed = 3
BM_2d_df_3 = pd.read_csv('trajectories/brownian_3.csv')

# Load existing trajectories
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories
CRW9_2d_df_6 = pd.read_csv('trajectories/crw_6_9.csv') 

In [ ]:
# Compute path length
## start - Add your code here
dis_bm2_s3 = pd.DataFrame([{"dis": 0}])
dis_bm2_s6 = pd.DataFrame([{"dis": 0}])
dis_crw9_s6 = pd.DataFrame([{"dis": 0}])

for i in range(1, BM_2d_df_3.shape[0]):
    pointA = np.array((BM_2d_df_3.iloc[i-1].x_pos, BM_2d_df_3.iloc[i-1].y_pos)) 
    pointB = np.array((BM_2d_df_3.iloc[i].x_pos, BM_2d_df_3.iloc[i].y_pos))
    dis = round(np.sqrt(np.sum(np.square(pointA - pointB))), 3)
    dis_bm2_s3 = pd.concat([dis_bm2_s3, pd.DataFrame([{"dis": dis_bm2_s3.iloc[i-1].dis + dis}])], ignore_index=True)

for i in range(1, BM_2d_df_6.shape[0]):
    pointA = np.array((BM_2d_df_6.iloc[i-1].x_pos, BM_2d_df_6.iloc[i-1].y_pos)) 
    pointB = np.array((BM_2d_df_6.iloc[i].x_pos, BM_2d_df_6.iloc[i].y_pos))
    dis = round(np.sqrt(np.sum(np.square(pointA - pointB))), 3)
    dis_bm2_s6 = pd.concat([dis_bm2_s6, pd.DataFrame([{"dis": dis_bm2_s6.iloc[i-1].dis + dis}])], ignore_index=True)

for i in range(1, CRW9_2d_df_6.shape[0]):
    pointA = np.array((CRW9_2d_df_6.iloc[i-1].x_pos, CRW9_2d_df_6.iloc[i-1].y_pos)) 
    pointB = np.array((CRW9_2d_df_6.iloc[i].x_pos, CRW9_2d_df_6.iloc[i].y_pos))
    dis = round(np.sqrt(np.sum(np.square(pointA - pointB))), 3)
    dis_crw9_s6 = pd.concat([dis_crw9_s6, pd.DataFrame([{"dis": dis_crw9_s6.iloc[i-1].dis + dis}])], ignore_index=True)
## end - Add your code here


In [ ]:
# Plotting
# Init figure
fig_path_length = go.Figure()

# First trace BM1
## start - Add your code here
fig_path_length.add_trace(
        go.Scatter(
            x = np.arange(len(dis_bm2_s3)),
            y = dis_bm2_s3.dis,
            line = dict(width=2),
            mode = 'lines',
            name = 'BM speed:3',
            showlegend = True
        )
    )
## end - Add your code here


# Second trace BM2
## start - Add e your code here
fig_path_length.add_trace(
        go.Scatter(
            x = np.arange(len(dis_bm2_s3)),
            y = dis_bm2_s6.dis,
            line = dict(width=4, color='rgb(35,35,35)'),
            mode = 'lines',
            name = 'BM speed:6',
            showlegend = True
        )
    )
## end - Add your code here


# Third trace CRW
fig_path_length.add_trace(
        go.Scatter(
            x = np.arange(len(dis_bm2_s3)),
            y = dis_crw9_s6.dis,
            line = dict(width=2, dash='dash', color='magenta'),
            mode = 'lines',
            name = 'CRW speed:6 c:0.9',
            showlegend = True
        )
    )
## start - Add your code here
    
## end - Add your code here

fig_path_length.update_layout(title_text='Random Walks - Path Length',
                          autosize=False,
                          width=800,
                          height=400
                          )
fig_path_length.show()


## Actividad 2: Mean Squared Displacement - (Brownian vs CRW)

* Cargar trayectorias en **Pandas** Data Frame
* Guardar metricas en **Pandas** Data Frame
* Visualizar con **plotly**

In [ ]:
# Load existing trajectories
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [ ]:
# Show trajectories
# Init figure
fig_3d = go.Figure()

# Plot trajectory in 3-D space
fig_3d.add_trace(
    go.Scatter3d(x = BM_2d_df_6.x_pos,
                 y = BM_2d_df_6.y_pos,
                 z = BM_2d_df_6.index,
                 marker = dict(size=2),
                 line = dict(color='blue', width=2),
                 mode = 'lines',
                 name = 'BM 2d',
                 showlegend = True))


fig_3d.add_trace(
    go.Scatter3d(x = CRW_2d_df_9.x_pos,
                 y = CRW_2d_df_9.y_pos,
                 z = CRW_2d_df_9.index,
                 marker = dict(size=2),
                 line = dict(color='red', width=2),
                 mode = 'lines',
                 name = 'CRW 2d',
                 showlegend = True))

fig_3d.show()

In [41]:
# Empty MSD_BM
MSD_BM = np.empty(shape=(0))

# MSD for BM_2d_df_6
for tau in range(1,BM_2d_df_6.shape[0]):
    ## start - Add your code here
    MSD_BM = np.array([distance.euclidean(BM_2d_df_6.iloc[i-tau], BM_2d_df_6.iloc[i])
        for i in range(tau, BM_2d_df_6.shape[0], 1)])
    displacement = MSD_BM.mean() ** 2
    ## end - Add your code here

# Empty MSD_CRW
MSD_CRW = np.empty(shape=(0))
# MSD for CRW_2d_df_9
for tau in range(1,CRW_2d_df_9.shape[0]):
    ## start - Add your code here
    MSD_CRW = np.array([distance.euclidean(CRW_2d_df_9.iloc[i-tau], CRW_2d_df_9.iloc[i])
        for i in range(tau + 1, CRW_2d_df_9.shape[0])])
    displacement = round(MSD_CRW.mean() ** 2, 4)
    print(displacement)
    ## end - Add your code here
    
# Save metrics to Dataframe
## start - Add your code here
    
## end - Add your code here

# Write to csv
## start - Add your code here
    
## end - Add your code here

36.0
135.1079
292.237
499.8125
753.4602
1047.5761
1380.0944
1746.7679
2143.5668
2566.7315
3012.5947
3478.0956
3961.5602
4463.5982
4981.5988
5515.852
6064.2669
6625.2511
7196.1633
7775.7526
8362.7499
8957.8046
9558.8785
10162.9104
10768.1695
11374.9954
11983.8937
12597.1673
13218.229
13842.7141
14470.8082
15103.2316
15738.337
16377.4347
17021.6248
17674.5685
18336.0214
19007.0719
19687.5957
20377.9668
21078.4458
21788.7383
22508.436
23236.587
23971.8645
24713.5936
25460.7032
26212.672
26968.7248
27728.7468
28492.2384
29259.6609
30031.4093
30807.9124
31588.8234
32374.0247
33163.7689
33958.1962
34757.5337
35561.6605
36370.0543
37182.5448
37999.2166
38820.0365
39645.8098
40478.2933
41319.3124
42171.0039
43037.3165
43915.5607
44796.8138
45678.1842
46558.1538
47440.2633
48324.4001
49209.9023
50096.863
50980.944
51861.9665
52740.03
53615.227
54486.4771
55352.81
56213.5764
57068.822
57918.9237
58763.1969
59601.4047
60435.6361
61265.2795
62095.0659
62925.5971
63754.9338
64583.087
65411.4477
662

KeyboardInterrupt: 

In [44]:
MSD_CRW = np.empty(shape=(0))
# MSD for CRW_2d_df_9
for tau in range(1,CRW_2d_df_9.shape[0]):
    ## start - Add your code here
    MSD_CRW = np.array([distance.euclidean(CRW_2d_df_9.iloc[i-tau], CRW_2d_df_9.iloc[i])
        for i in range(tau, CRW_2d_df_9.shape[0])])
    displacement = round(MSD_CRW.mean() ** 2, 4)

36.0
135.1079
292.237
499.8125
753.4602
1047.5761
1380.0944
1746.7679
2143.5668
2566.7315
3012.5947
3478.0956
3961.5602
4463.5982
4981.5988
5515.852
6064.2669
6625.2511
7196.1633
7775.7526
8362.7499
8957.8046
9558.8785
10162.9104
10768.1695
11374.9954
11983.8937
12597.1673
13218.229
13842.7141
14470.8082
15103.2316
15738.337
16377.4347
17021.6248
17674.5685
18336.0214
19007.0719
19687.5957
20377.9668
21078.4458
21788.7383
22508.436
23236.587
23971.8645
24713.5936
25460.7032
26212.672
26968.7248
27728.7468
28492.2384
29259.6609
30031.4093
30807.9124
31588.8234
32374.0247
33163.7689
33958.1962
34757.5337
35561.6605
36370.0543
37182.5448
37999.2166
38820.0365
39645.8098
40478.2933
41319.3124
42171.0039
43037.3165
43915.5607
44796.8138
45678.1842
46558.1538
47440.2633
48324.4001
49209.9023
50096.863
50980.944


KeyboardInterrupt: 

In [ ]:
# Init figure
fig_path_length = go.Figure()

# first trace MSD_BM
## start - Add your code here
    
## end - Add your code here


# Second trace MSD_CRW
## start - Add your code here
    
## end - Add your code here


fig_path_length.show()